In this notebook we provide some useful aggregations

In [1]:
# import libraries

import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px

In [2]:
# establish connection with database

engine = create_engine('sqlite:///../db/invoicedb')
connection = engine.connect()

In [3]:
def create_bar_plot(df:pd.DataFrame, x:str, y:str, text:str, title:str, xtitle, ytitle) -> None:
    '''

    '''
    fig = px.bar(df, x=x, y=y, text=text, title=title)

    fig.update_layout(height=600,
                    xaxis_title=xtitle,
                    yaxis_title=ytitle)
    fig.show()

Top 10 selling products

In [4]:
# query to find the top 10 selling products

sql_query = '''
    
    SELECT
        s.Description,
        SUM(i.Quantity) AS TOTAL_QUANTITY
    FROM InvoiceFact i
    INNER JOIN StockDim s 
    ON s.StockCode = i.StockCode
    GROUP BY s.Description
    ORDER BY TOTAL_QUANTITY DESC
    LIMIT 10;
    
'''

# Execute the query and fetch the results into a Pandas DataFrame
top_10_selling_prods = pd.read_sql_query(sql_query, connection)

# print the df with the results
top_10_selling_prods

,Description,TOTAL_QUANTITY
0,pack of 72 retrospot cake cases,56775
1,white hanging heart tlight holder,55751
2,world war 2 gliders asstd designs,54130
3,jumbo bag red retrospot,48629
4,brocade ring purse,47430
5,assorted colour bird ornament,44000
6,60 teatime fairy cake cases,35480
7,pack of 60 pink paisley cake cases,30753
8,small popcorn holder,28435
9,black and white paisley flower mug,25679


In [5]:
create_bar_plot(top_10_selling_prods,
                'Description',
                'TOTAL_QUANTITY',
                'TOTAL_QUANTITY',
                'Top 10 selling products',
                "Product",
                "Quantity")

Last 10 selling products

In [6]:
# query to find the bottom 10 selling products

sql_query = '''
    
    SELECT
        s.Description,
        SUM(i.Quantity) AS TOTAL_QUANTITY
    FROM InvoiceFact i
    INNER JOIN StockDim s 
    ON s.StockCode = i.StockCode
    WHERE Quantity > 0
    GROUP BY s.Description
    ORDER BY TOTAL_QUANTITY ASC
    LIMIT 10;
    
'''

# Execute the query and fetch the results into a Pandas DataFrame
bottom_10_selling_prods = pd.read_sql_query(sql_query, connection)

# print the df with the results
bottom_10_selling_prods

,Description,TOTAL_QUANTITY
0,3 birds canvas screen,1
1,6 hook jewel stand lilac dress,1
2,amber bertie glass bead bag charm,1
3,amber crystal drop earrings,1
4,baroque butterfly earrings crystal,1
5,baroque butterfly earrings red,1
6,black diamond cluster earrings,1
7,black drop crystal necklace,1
8,black drop earrings w long beads,1
9,black rnd bulletkeep clean bin,1


In [7]:
create_bar_plot(bottom_10_selling_prods,
                'Description',
                'TOTAL_QUANTITY',
                'TOTAL_QUANTITY',
                'Bottom 10 selling products',
                "Product",
                "Quantity")

Customers with most product buys

In [8]:
# query to find the top 10 customers by quantity

sql_query = '''
    
    SELECT
        CD.CustomerID,
        SUM(Quantity) AS TotalQuantity
    FROM
        InvoiceFact IF
    JOIN
        CustomerDim CD ON IF.CustomerID = CD.CustomerID
    GROUP BY
        CD.CustomerID
    ORDER BY
        TotalQuantity DESC
    LIMIT 10;
    
'''

# Execute the query and fetch the results into a Pandas DataFrame
top10_customers_quant = pd.read_sql_query(sql_query, connection)

# print the df with the results
top10_customers_quant

,CustomerID,TotalQuantity
0,13902,218088
1,14646,168464
2,13694,123642
3,18102,123504
4,14156,106865
5,13687,87167
6,17940,75825
7,14911,66508
8,17511,54901
9,16754,54598


In [9]:
create_bar_plot(top10_customers_quant,
                'CustomerID',
                'TotalQuantity',
                'TotalQuantity',
                'Top 10 buyers',
                "CustomerID",
                "Quantity")

Customer who spends the most

In [10]:
# query to find the top 10 customers that spends the most

sql_query = '''
    
    SELECT
        CD.CustomerID,
        CAST(SUM(CASE WHEN IF.Quantity >= 0 THEN IF.Price * IF.Quantity ELSE 0 END) AS INTEGER) AS TotalAmount

    FROM
        InvoiceFact IF
    JOIN
        CustomerDim CD ON IF.CustomerID = CD.CustomerID
    GROUP BY
        CD.CustomerID
    ORDER BY
        TotalAmount DESC
    LIMIT 10;
    
'''

# Execute the query and fetch the results into a Pandas DataFrame
top10_customers_amount = pd.read_sql_query(sql_query, connection)

# print the df with the results
top10_customers_amount

,CustomerID,TotalAmount
0,18102,349164
1,14646,247613
2,14156,187371
3,14911,140900
4,13694,131443
5,17511,84541
6,15061,83282
7,16684,80489
8,16754,65500
9,13089,57885


In [11]:
create_bar_plot(top10_customers_amount,
                'CustomerID',
                'TotalAmount',
                'TotalAmount',
                'Top 10 buyers who spends the most',
                "CustomerID",
                "TotalAmount")

Top 10 products with most returns

In [12]:
# query to find the top 10 products with more returns

sql_query = '''
    
    SELECT
        SD.Description,
        SUM(ABS(IF.Quantity)) AS TotalCacelations
    FROM InvoiceFact IF
    INNER JOIN StockDim SD
    ON IF.StockCode = SD.StockCode
    WHERE IF.Invoice LIKE 'C%'
    GROUP BY SD.StockCode
    ORDER BY TotalCacelations DESC
    LIMIT 10;
    
'''

# Execute the query and fetch the results into a Pandas DataFrame
top10_product_cncl = pd.read_sql_query(sql_query, connection)

# print the df with the results
top10_product_cncl

,Description,TotalCacelations
0,rotating silver angels tlight hldr,9363
1,set6 fruit salad paper cups,7128
2,set6 fruit salad paper plates,7008
3,pop art pen case pens,5184
4,black silver flower tlight holder,5040
5,multicolour spring flower mug,4993
6,teatime pen case pens,4632
7,white bird garden design mug,4320
8,s4 blue round decoupage boxes,3940
9,the king gift bag,3744


In [13]:
create_bar_plot(top10_product_cncl,
                'Description',
                'TotalCacelations',
                'TotalCacelations',
                'Top 10 products with more returns',
                "Description",
                "TotalCacelations")

Top 10 Countries per amount spent

In [14]:
# query to find the top 10 countries that spends the most

sql_query = '''
    
    SELECT
        CD.Country,
        CAST(SUM(CASE WHEN IF.Quantity >= 0 THEN IF.Price * IF.Quantity ELSE 0 END) AS INTEGER) AS TotalAmount

    FROM
        InvoiceFact IF
    JOIN
        CustomerDim CD ON IF.CustomerID = CD.CustomerID
    GROUP BY
        CD.Country
    ORDER BY
        TotalAmount DESC
    LIMIT 10;
    
'''

# Execute the query and fetch the results into a Pandas DataFrame
top10_countries_amount = pd.read_sql_query(sql_query, connection)

# print the df with the results
top10_countries_amount

,Country,TotalAmount
0,United Kingdom,7325554
1,EIRE,335391
2,Netherlands,266061
3,Germany,183020
4,France,131318
5,Denmark,50086
6,Sweden,49216
7,Spain,42908
8,Switzerland,42545
9,Australia,30484


In [15]:
create_bar_plot(top10_countries_amount,
                'Country',
                'TotalAmount',
                'TotalAmount',
                'Top 10 countries per amount spent',
                "Country",
                "TotalAmount")

Sales per month

In [16]:
# query to find the monthly sales

sql_query = '''
    
    SELECT

        D.Month || '/' ||
        D.Year AS Month,
        CAST(SUM(CASE WHEN IF.Quantity >= 0 THEN IF.Price * IF.Quantity ELSE 0 END) AS INTEGER) AS TotalAmount
    FROM
        InvoiceFact IF
    JOIN
        DateDim D ON IF.DateID = D.DateID
    GROUP BY
        D.Year,
        D.Month
    ORDER BY
        D.Year,
        D.Month;
    
'''

# Execute the query and fetch the results into a Pandas DataFrame
monthly_amnt = pd.read_sql_query(sql_query, connection)

# print the df with the results
monthly_amnt

,Month,TotalAmount
0,12/2009,678029
1,1/2010,537341
2,2/2010,497937
3,3/2010,665973
4,4/2010,585149
5,5/2010,592734
6,6/2010,629540
7,7/2010,581487
8,8/2010,594561
9,9/2010,805544


In [17]:
create_bar_plot(monthly_amnt,
                'Month',
                'TotalAmount',
                'TotalAmount',
                'Monthly sales',
                'Month',
                "TotalAmount")

In [18]:
# query to find invoices per hour 

sql_query = '''
    
    SELECT
        D.Hour AS Hour,
        COUNT(DISTINCT IF.Invoice) AS Invoices
    FROM
        InvoiceFact IF
    INNER JOIN
        DateDim D ON IF.DateID = D.DateID
    GROUP BY
    D.Hour
    ORDER BY D.Hour ASC;
    
'''

# Execute the query and fetch the results into a Pandas DataFrame
num_inv_per_hour = pd.read_sql_query(sql_query, connection)

# print the df with the results
num_inv_per_hour

,Hour,Invoices
0,7,79
1,8,536
2,9,1409
3,10,2637
4,11,2904
5,12,3630
6,13,3359
7,14,2756
8,15,2406
9,16,1600


In [19]:
fig = px.line(data_frame=num_inv_per_hour, x='Hour', y='Invoices', title="Num of Invoices per hour")
fig.show()


Top 5 customers in purchases per year

In [20]:
# query to find top 5 customers per year

sql_query = '''
    
    WITH InvoiceAmounts AS (
        SELECT
            d.Year,
            c.CustomerID,
            CAST(SUM(CASE WHEN i.Quantity >= 0 THEN i.Price * i.Quantity ELSE 0 END) AS INTEGER) AS TotalAmountSpent
        FROM
            InvoiceFact AS i
        JOIN
            DateDim AS d ON i.DateID = d.DateID
        JOIN
            CustomerDim AS c ON i.CustomerID = c.CustomerID
        GROUP BY
            d.Year,
            c.CustomerID
    ),
    RankedCustomers AS (
        SELECT
            Year,
            CustomerID,
            TotalAmountSpent,
            ROW_NUMBER() OVER (PARTITION BY Year ORDER BY TotalAmountSpent DESC) AS CustomerRank
        FROM
            InvoiceAmounts
    )
    SELECT
        Year,
        CustomerID,
        TotalAmountSpent,
        CustomerRank
    FROM
        RankedCustomers
    WHERE
        CustomerRank <= 5
    ORDER BY
        Year,
        CustomerRank;
    
'''

# Execute the query and fetch the results into a Pandas DataFrame
top_5_cust_per_year = pd.read_sql_query(sql_query, connection)

# print the df with the results
top_5_cust_per_year

,Year,CustomerID,TotalAmountSpent,CustomerRank
0,2009,18102,41005,1
1,2009,13694,20427,2
2,2009,14646,14591,3
3,2009,15061,14312,4
4,2009,14156,10951,5
5,2010,18102,308158,1
6,2010,14646,233022,2
7,2010,14156,176420,3
8,2010,14911,134684,4
9,2010,13694,111015,5


In [21]:
# close connection with database

connection.close()